In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import os
cwd = os.getcwd()
cwd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('./drive/MyDrive/DS/WA_Fn-UseC_-HR-Employee-Attrition.csv')
df

### Encoding

In [ ]:
df.drop(['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'], axis='columns', inplace=True)

In [ ]:
df['Age_re'] = df['Age']
df = df.drop('Age',axis=1)

In [ ]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
df['BusinessTravel'] = le.fit_transform(df.BusinessTravel.values)
df['Department'] = le.fit_transform(df.Department.values)
df['Gender'] = le.fit_transform(df.Gender.values)
df['JobRole'] = le.fit_transform(df.JobRole.values)
df['MaritalStatus'] = le.fit_transform(df.MaritalStatus.values)
df['OverTime'] = le.fit_transform(df.OverTime.values)
df['EducationField'] = le.fit_transform(df.EducationField.values)
df


In [ ]:
from sklearn.preprocessing import LabelEncoder

for column in df.columns:
    if df[column].dtype == np.number:
        continue 
    df[column] = LabelEncoder().fit_transform(df[column])

### Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df.to_numpy())
df_scaled = pd.DataFrame(df_scaled)
 
print("Scaled Dataset Using MinMaxScaler")
df_scaled

### Specify Predictor/Target Variables

In [ ]:
X = df.drop('Attrition', axis=1)
y = df['Attrition']

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_train, y_train = sm.fit_resample(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, bootstrap=False)

                                    
rf_clf.fit(X_train, y_train)
print('Training accuracy:',np.mean(rf_clf.predict(X_train)==y_train)*100)
print('Test accuracy:',np.mean(rf_clf.predict(X_test)==y_test)*100)

In [ ]:
importance_vals = rf_clf.feature_importances_
plt.rcParams['figure.figsize'] = [10,10]
indices = np.argsort(importance_vals)[::-1]

plt.figure()
plt.title('Random forest impurity-based feature importance')
plt.bar(range(X_train.shape[1]), importance_vals[indices])

plt.xticks(range(X_train.shape[1]), df.columns[1:][indices], rotation = 90)
plt.xlim([-1, X_train.shape[1]])
plt.ylim([0,0.2])

#plt.tight_layout()
plt.show()


In [ ]:
## How much a performance will drop if we shuffle the column 